In [10]:
import pandas as pd

book_rating = pd.read_csv(r'./Goodreads.csv', encoding='ISO-8859-1')

# book_rating.head()

In [17]:
import os
import pandas as pd

file_path = 'Goodreads.csv'
print(os.path.abspath(file_path))


c:\Users\ASYIKIN\OneDrive\Desktop\PYTHON FOR DATA ANALYSIS\PROJECTS\Book\Goodreads.csv


In [2]:
book_rating.columns

Index(['ID', 'Title', 'Author', 'Fiction', 'Genre', 'Gender', 'Origin',
       'Language', 'Pages', 'Year Published', 'Centuries', 'Nobel Prize',
       'Avg_rating', 'Num_rating', 'Num_reviews', 'textblob_sentiment',
       'vader_sentiment', 'URL'],
      dtype='object')

# Descriptive Statistics

- I had to chnage the data type of num_rating and num_reviews to integers as their values for descriptive stats werent shown - they were classified as 'object'


In [3]:
#book_rating.dtypes

# book_rating[['Avg_rating', 'Num_rating', 'Num_reviews']].describe()

In [4]:
import plotly.express as px

# distr_avg_rating = px.histogram(book_rating, x='Num_rating', nbins=20, title='Distribution of Number of Ratings')
# distr_avg_rating.show()

# distr_avg_rating = px.histogram(book_rating, x='Num_reviews', nbins=20, title='Distribution of Number of Reviews')
# distr_avg_rating.show()

In [5]:
#Top Performers
top_books = book_rating.nlargest(15, 'Num_rating')[['Title', 'Author', 'Num_rating']]

fig_top_books = px.bar(
    top_books, x='Num_rating', y='Title', orientation='h',
    color='Num_rating', color_continuous_scale='Viridis',
    title='Top 15 most popular books by ratings on Goodreads',
    text=top_books['Author'],
    template='plotly_dark'
)

fig_top_books.update_layout(
    title_x = 0.5,
    title_font=dict(size=15, color='yellow'),
    width=780,
    coloraxis_colorbar=dict(
        title='',  # Remove colorbar title
        len=0.9,   # Reduce colorbar height (scale between 0-1)
        thickness=10,  # Narrower colorbar
    ),
    yaxis={
        'categoryorder': 'total ascending',
        'tickfont': {
            'size': 8,  # y axis tick font size
            'color': 'white'
        }
    },
    margin=dict(
        l=10,  # Left margin
        r=10,  # Right margin
        t=50,  # Top margin (space for title)
        b=10   # Bottom margin
    ),
)

fig_top_books.write_html("./charts/popular_books.html")

fig_top_books.show()

In [6]:
#top 15 most reviewed books
top_books_rev = book_rating.nlargest(15, 'Num_reviews')[['Title', 'Author', 'Num_reviews']]

fig_top_books_rev = px.bar(
    top_books_rev, x='Num_reviews', y='Title', orientation='h',
    color='Num_reviews', color_continuous_scale='Viridis',
    title='Top 15 most reviewed Books on Goodreads',
    text=top_books_rev['Author'],
    template='plotly_dark'
)

fig_top_books_rev.update_layout(
    yaxis={'categoryorder': 'total ascending', 'title': ''},
    title_x=0.5,
    coloraxis_colorbar=dict(title='Reviews (in thousands)')
)

fig_top_books_rev.update_traces(
    textposition='inside',
    textfont=dict(size=12),
    showlegend=False
    )
fig_top_books_rev.show()

# Genre

* The most prominent genres in my dataset can be categorized into major groups:
    - Faith, Philosophy, Autobiography, Science, Psychology, History, Self-help

* The minor groups whihc has less than 5 counts:
    - Economy, Dystopian, Biography, Photography, Feminism, Politics, Humor, Drama.

In [7]:
unique_genres = book_rating['Genre'].unique()
unique_genres

array(['Dystopian', 'History', 'Faith', 'Psychology', 'Politics',
       'Biography', 'Science', 'Autobiography', 'Self-help', 'Philosophy',
       'Economy', 'Photography', 'Humor', 'Drama', 'Feminism'],
      dtype=object)

In [8]:
genre_counts = book_rating['Genre'].value_counts()
genre_counts

Genre
Faith            19
Philosophy       17
Autobiography    14
Psychology       12
Science          11
History           8
Self-help         5
Economy           4
Dystopian         2
Biography         2
Photography       2
Politics          1
Humor             1
Drama             1
Feminism          1
Name: count, dtype: int64

In [9]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

#define minor genress
minor_genres = ['Economy', 'Dystopian', 'Biography', 'Photography', 'Feminism', 'Politics', 'Humor', 'Drama']

#Create grouped_genre column
book_rating['Grouped_Genre'] = book_rating['Genre'].apply(lambda x: 'Others' if x in minor_genres else x)

#initialize dash app 
app = Dash(__name__)

#define app layut 
app.layout = html.Div(style={'backgroundColor': '#1e1e1e'}, children=[
    html.Div(style={'position': 'relative', 'width': '100%', 'height':'500px'}, children=[
        
        #dropdown positioned absolutely within chart container
        html.Div(style={'position': 'absolute', 'top':'20px', 'left': '20px', 'z-index':'1', 'width':'200px', 'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'}, children=[
            html.Label('Select Genre', style={'color': 'salmon', 'fontSize': 14, 'display': 'block', 'textAlign': 'center', 'marginRight': '50px'}),
            dcc.Dropdown(
                id='genre-dropdown',
                options=[{'label': genre, 'value': genre} for genre in book_rating['Grouped_Genre'].unique()],
                value = 'Faith',
                style={'width': '100px', 'height':'30px', 'display': 'inline-block', 'fontSize': 12, 'marginRight': '50px'},
                clearable=False
            )
        ]),
         dcc.Graph(id='genre-bar-chart', style={'height': '100%'})
    ])     
])

#define callback to update the bar chart
@app.callback(
    Output('genre-bar-chart', 'figure'),
    Input('genre-dropdown', 'value')
)

def update_chart(selected_genre):
    genre_books = book_rating[book_rating['Grouped_Genre'] == selected_genre].sort_values(by='Num_rating')

    fig = go.Figure()
    
    fig.add_trace(
        go.Bar(
            x=genre_books['Num_rating'],
            y=genre_books['Title'],
            orientation='h',
            text=genre_books['Title'],
            textposition='inside',
            hovertext=genre_books['Author'],
            hovertemplate=(
                "<b>%{text}</b><br>"  # Title is bold and at the top
                "Author: %{hovertext}<br>"  # Author comes after the title
                "<extra></extra>"  # Removes extra hover information (like the legend)
            ),
            marker=dict(
                color=genre_books['Num_rating'],
                colorscale='Viridis', 
                showscale=True,
                colorbar=dict(
                    title='',
                    thickness=15
                )
            )
        )
    )
    
    title_text = f'Goodreads Ratings: <span style="color: fuchsia";>{selected_genre}</span>'
    
    fig.update_layout(
        template='plotly_dark',
        title=title_text,
        title_x=0.5,
        title_font_size=15,
        title_font_color= 'yellow',
        xaxis_title='Number of Ratings (Log Scale)',
        yaxis_title='Book Title',
        xaxis_title_font=dict(size=10),
        width=800,
        height=520,
        yaxis=dict(showticklabels=False),  # Hide y-axis labels
        margin=dict(l=10, r=10, t=70, b=10), 
    )
    
    fig.update_xaxes(type="log")

    return fig

# default browser: http://127.0.0.1:8058

if __name__ == '__main__':
    app.run_server(debug=True, port=8058)